In [5]:
import veryfi
import os
import json
import re
import pandas as pd

# Configuración de credenciales de Veryfi
client_id = 'vrfhjAs5kPIee2Yvy7F0osVV5MX6zXKFTriJyLS'
client_secret = 'NJoXZcE25MBxv97merROYW5CvYBJLmNwrGDoYEWzv970T0fyeE2pxAXlP93huX46Bf1NbgM6jGzVb3LzSrTWZYDEGcyaruHwfWAr75EfA87rcoo29NLYzWRWJdogbc1N'
username = 'juanpamon.13'
api_key = 'f604ee697fd73c0ba1bb99a536bcde0b'

client = veryfi.Client(client_id, client_secret, username, api_key)

# Directorios
input_folder = "Documents"
veryfi_output_folder = "veryfi_json_v3"
processed_output_folder = "output_v3"

# Crear las carpetas de salida si no existen
os.makedirs(veryfi_output_folder, exist_ok=True)
os.makedirs(processed_output_folder, exist_ok=True)

def extract_invoice_data(ocr_text):
    """
    Extrae la información clave de la factura desde el texto OCR.
    """
    invoice_data = {}

    # Vendor Name
    vendor_match = re.search(r'Please make payments to:\s*(.+)', ocr_text)
    invoice_data["vendor_name"] = vendor_match.group(1).strip() if vendor_match else "Unknown"

    # Vendor Address (Fijo)
    vendor_address_match = re.search(r"\n([\w\s,]+ \d{5}(?:-\d{4})?)\nPO Box", ocr_text)
    if vendor_address_match:
        address = vendor_address_match.group(1).strip()
        address_parts = address.split()[-3:]  # Tomar las últimas tres partes (por ejemplo: ciudad, estado y código postal)
        invoice_data["vendor_address"] = " ".join(address_parts)
    else:
        invoice_data["vendor_address"] = "Unknown"

    # Invoice Number y Bill To Name
    invoice_match = re.search(r'Invoice No\.\n\t(\d{2}/\d{2}/\d{2})\t(\d{2}/\d{2}/\d{2})\t([\d]+)\n\n(.+)', ocr_text)
    if invoice_match:
        invoice_data["date"] = invoice_match.group(1)  # Primera fecha después de Invoice No.
        invoice_data["invoice_number"] = invoice_match.group(3)  # Número de factura
        invoice_data["bill_to_name"] = invoice_match.group(4).strip()  # Nombre después de Invoice No.
    else:
        invoice_data["date"] = "Unknown"
        invoice_data["invoice_number"] = "Unknown"
        invoice_data["bill_to_name"] = "Unknown"

    # --- Paso 1: Extraer la sección de precios ---
    # Usamos patrones con flag MULTILINE para identificar la línea de cabecera y la de totales.
    start_pattern = r"^Description.*Quantity.*Rate.*Amount\s*$"
    end_pattern = r"^\tTotal\s+USD"
    
    match_start = re.search(start_pattern, ocr_text, flags=re.MULTILINE)
    match_end = re.search(end_pattern, ocr_text, flags=re.MULTILINE)
    
    if match_start and match_end:
        price_section = ocr_text[match_start.end():match_end.start()]
    else:
        raise ValueError("No se pudo identificar la sección de lista de precios.")

    # Para mantener la nomenclatura anterior, asignamos el fragmento extraído a oct_text
    oct_text = price_section

    # --- Paso 2: Procesar oct_text para reconstruir la tabla de 4 columnas ---
    # La tabla se compone de:
    #   - Columna 1: Descripción (puede ocupar varias líneas)
    #   - Columna 2: Cantidad
    #   - Columna 3: Precio
    #   - Columna 4: Total

    rows = []
    current_row = None

    for line in oct_text.splitlines():
        # Si la línea contiene tabuladores, se asume que es el comienzo de un registro
        if "\t" in line:
            tokens = line.split("\t")
            # Si la línea no tiene al menos 4 tokens, se asume que es parte de la descripción
            if len(tokens) < 4:
                if current_row is not None:
                    current_row[0] += " " + line.strip()
                continue
            # Si ya se estaba construyendo un registro, se agrega a la lista
            if current_row is not None:
                rows.append(current_row)
            # Se asume que los últimos 3 tokens son Cantidad, Precio y Total
            description_tokens = tokens[:-3]
            description = " ".join(tok.strip() for tok in description_tokens if tok.strip())
            col1 = tokens[-3].strip()  # Cantidad
            col2 = tokens[-2].strip()  # Precio
            col3 = tokens[-1].strip()  # Total
            current_row = [description, col1, col2, col3]
        else:
            # Si la línea no contiene tabuladores, es continuación de la descripción
            if current_row is not None:
                current_row[0] += " " + line.strip()

    if current_row is not None:
        rows.append(current_row)

    df = pd.DataFrame(rows, columns=["Descripción", "Col1", "Col2", "Col3"])

    # --- Paso 3: Extraer los Items a partir del DataFrame ---
    items = []
    for index, row in df.iterrows():
        try:
            quantity = float(row["Col1"].replace(',', ''))
        except (AttributeError, ValueError):
            quantity = 0.0

        try:
            price = float(row["Col2"].replace(',', ''))
        except (AttributeError, ValueError):
            price = 0.0

        try:
            total = float(row["Col3"].replace(',', ''))
        except (AttributeError, ValueError):
            total = 0.0

        items.append({
            "sku": "Unknown",  # No se extrajo SKU en este nuevo procesamiento
            "description": row["Descripción"].strip(),
            "quantity": quantity,
            "tax_rate": None,  # No está presente en la factura
            "price": price,
            "total": total
        })

    invoice_data["items"] = items
    return invoice_data

# Procesar todos los archivos PDF en la carpeta Documents
for file_name in os.listdir(input_folder):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(input_folder, file_name)
        print(f"Procesando: {file_name}")
        
        # Extraer datos con Veryfi
        #json_result = client.process_document(file_path)
        
        # Guardar el resultado en la carpeta veryfi_json_v3
        veryfi_output_file = os.path.join(veryfi_output_folder, f"{file_name}.json")
        #with open(veryfi_output_file, "w", encoding="utf-8") as f:
        #    json.dump(json_result, f, ensure_ascii=False, indent=4)
        
        print(f"Resultado guardado en: {veryfi_output_file}")
        
        # Procesar JSON y extraer datos clave
        with open(veryfi_output_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            ocr_text = data.get("ocr_text", "")
            invoice_info = extract_invoice_data(ocr_text)
        
        # Guardar el resultado procesado en la carpeta output_v3
        processed_output_file = os.path.join(processed_output_folder, f"{file_name}.json")
        with open(processed_output_file, "w", encoding="utf-8") as f:
            json.dump(invoice_info, f, ensure_ascii=False, indent=4)
        
        print(f"Datos procesados guardados en: {processed_output_file}")

print("Procesamiento completado.")


Procesando: synth-switch_v5-14.pdf
Resultado guardado en: veryfi_json_v3\synth-switch_v5-14.pdf.json
Datos procesados guardados en: output_v3\synth-switch_v5-14.pdf.json
Procesando: synth-switch_v5-4.pdf
Resultado guardado en: veryfi_json_v3\synth-switch_v5-4.pdf.json
Datos procesados guardados en: output_v3\synth-switch_v5-4.pdf.json
Procesando: synth-switch_v5-68.pdf
Resultado guardado en: veryfi_json_v3\synth-switch_v5-68.pdf.json
Datos procesados guardados en: output_v3\synth-switch_v5-68.pdf.json
Procesando: synth-switch_v5-7.pdf
Resultado guardado en: veryfi_json_v3\synth-switch_v5-7.pdf.json
Datos procesados guardados en: output_v3\synth-switch_v5-7.pdf.json
Procesando: synth-switch_v5-79.pdf
Resultado guardado en: veryfi_json_v3\synth-switch_v5-79.pdf.json
Datos procesados guardados en: output_v3\synth-switch_v5-79.pdf.json
Procesamiento completado.


In [2]:
ocr_text

'Invoice\n\tPage 1 of 2\nswitch\tDallas, TX 75267-4592\nPO Box 674592\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/06/24\t05/06/24\t055205954\n\nMicro Merchant Systems, Inc.\n25 Thomson Place\n4th Floor\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of November\nO-9589055\t\t\t\t\tPO-0I2U-1292\nDescription\t\t\t\tQuantity\tRate\tAmount\nInstallation of Cross Connect | 395 Gbps Fiber to AE9qC3\t\t\t\t579.10\t1,750.30\t1,013,598.73\nTransport | 71 Gbps Fiber to 04pBZ (02FDIWHG) (07/2023)\t\t\t\t5,519.81\t5,201.91\t28,713,554.84\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (0902986589) (03/2023 | 10 Gbps Fiber\t8,360.15\t930.70\t7,780,791.61\nto TcUjOmG (N0SBM4F4) (03/2023)\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (5659868303) (01/2024|10 Gbps Fiber\t2,983.62\t504.07\t1,503,953.33\nto Ei9sXj (4UV4KE73) (01/2024)\nTransport | 773 Gbps Fiber to 6YDZ35e (BF3EE94B) (06/2023)\t\t\t2,999.36\t6,741.52\t20,220,245.43\nCarrier Taxes for Transport | 522 Gbps Wavelength